In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import scipy.constants as cst
from lammpsrun import LAMMPS, Prism

from ase.lattice.cubic import BodyCenteredCubic
from ase.visualize import view
from ase.io.trajectory import Trajectory
from ase.io import write
from ase.build import *

homepath = "D:\\Cloud Storage\\GitRepositories\\LAMMPS_Simulation\\"
homepath

Populating the interactive namespace from numpy and matplotlib


'D:\\Cloud Storage\\GitRepositories\\LAMMPS_Simulation\\'

Per Prof. Nic Harrison,
$$ SE = \frac{E_{slab} - E_{bulk}}{2A}$$

In [2]:
Fe_slab_reaxff_params = {
    # Initialization
    "units": "real",
    "atom_style": "charge",
    "mass": ["1 55.845"],
    
    # Forcefield definition
    "pair_style": "reax/c NULL safezone 16",
    "pair_coeff": ['* * ' + 'ffield.reax.Fe_O_C_H' + ' Fe'],
    "neighbor": "2.0 bin",
    "neighbor_modify": "delay 10 check yes",
    
    # Run and Minimization
    "minimize": "1e-25 1e-25 10000 10000",
    "min_style": "cg",
    "timestep": 0.1,
    "fix": ["1 all box/relax xy 0.0 vmax 0.001",
            "2 all qeq/reax 1 0.0 10.0 1e-6 reax/c"]
}

Fe_bulk_reaxff_params = {
    # Initialization
    "units": "real",
    "atom_style": "charge",
    "mass": ["1 55.845"],
    
    # Forcefield definition
    "pair_style": "reax/c NULL safezone 16",
    "pair_coeff": ['* * ' + 'ffield.reax.Fe_O_C_H' + ' Fe'],
    "neighbor": "2.0 bin",
    "neighbor_modify": "delay 10 check yes",
    
    # Run and Minimization
    "minimize": "1e-25 1e-25 10000 10000",
    "min_style": "cg",
    "timestep": 0.1,
    "fix": ["1 all box/relax iso 0.0 vmax 0.001",
            "2 all qeq/reax 1 0.0 10.0 1e-6 reax/c"]
}

## Bulk Fe

In [15]:
view(Fe_bulk, viewer='ngl')

In [3]:
# Creating simulation box
Fe_bulk = bulk('Fe', 'bcc')

# Setting up simulation input
Fe_bulk_calc = LAMMPS(tmp_dir = homepath + "temp", 
                      parameters = Fe_bulk_reaxff_params,
                      keep_tmp_files=True, no_data_file=True)
Fe_bulk.set_calculator(Fe_bulk_calc)
Fe_bulk_calc.atoms = Fe_bulk.copy()
Fe_bulk_calc.prism = Prism(Fe_bulk_calc.atoms.get_cell())

# Writing input file
Fe_bulk_calc.write_lammps_in(lammps_in = homepath + "surface_Energy\\fe_bulk_reaxff_minimize.in",
                             lammps_trj = homepath + "surface_Energy\\fe_bulk_reaxff_minimize.lammpstrj")


In [ ]:
# Running calculations
!lmp_serial -in surface_Energy\fe_bulk_reaxff_minimize.in -log surface_Energy\fe_bulk_reaxff_minimize.log

In [4]:
# Reading output
Fe_bulk_calc.read_lammps_log(lammps_log = homepath + "surface_Energy\\fe_bulk_reaxff_minimize.log")
Fe_bulk_data = Fe_bulk_calc.thermo_content[-1]

# bcc(110) Fe

In [5]:
# Creating simulation box
Fe_bcc110 = bcc110('Fe', size=(1,1,128), vacuum=10.0)

# Setting up simulation input
Fe_bcc110_calc = LAMMPS(tmp_dir = homepath + "temp", 
                      parameters = Fe_slab_reaxff_params,
                      keep_tmp_files=True, no_data_file=True)
Fe_bcc110.set_calculator(Fe_bcc110_calc)
Fe_bcc110_calc.atoms = Fe_bcc110.copy()
Fe_bcc110_calc.prism = Prism(Fe_bcc110_calc.atoms.get_cell())

# Writing input file
Fe_bcc110_calc.write_lammps_in(lammps_in = homepath + "surface_Energy\\fe_bcc110_reaxff_minimize.in",
                               lammps_trj = homepath + "surface_Energy\\fe_bcc110_reaxff_minimize.lammpstrj")


In [ ]:
# Running calculations
!lmp_serial -in surface_Energy\fe_bcc110_reaxff_minimize.in -log surface_Energy\fe_bcc110_reaxff_minimize.log

In [6]:
# Reading output
Fe_bcc110_calc.read_lammps_log(lammps_log = homepath + "surface_Energy\\fe_bcc110_reaxff_minimize.log")
Fe_bcc110_data = Fe_bcc110_calc.thermo_content[-1]

In [14]:
Fe_bcc110_data = Fe_bcc110_calc.thermo_content[-1]

# Surface energy in Kcal/(mol A^2)
Fe_bcc110_SE_real = (Fe_bcc110_data['pe']/Fe_bcc110_data['atoms'] - Fe_bulk_data['pe']/Fe_bulk_data['atoms'])/(2*(Fe_bcc110_data['vol']/Fe_bcc110_data['lz']/Fe_bcc110_data['atoms']))

# Converting to J/m^2
Fe_bcc110_SE_SI = Fe_bcc110_SE_real * 4184 / cst.Avogadro * 10**20
print(Fe_bcc110_SE_SI)

1.7443148875538246


# bcc(100) Fe

In [7]:
# Creating simulation box
Fe_bcc100 = bcc100('Fe', size=(1,1,128), vacuum=10.0)

# Setting up simulation input
Fe_bcc100_calc = LAMMPS(tmp_dir = homepath + "temp", 
                      parameters = Fe_slab_reaxff_params,
                      keep_tmp_files=True, no_data_file=True, always_triclinic=True)
Fe_bcc100.set_calculator(Fe_bcc100_calc)
Fe_bcc100_calc.atoms = Fe_bcc100.copy()
Fe_bcc100_calc.prism = Prism(Fe_bcc100_calc.atoms.get_cell())

# Writing input file
Fe_bcc100_calc.write_lammps_in(lammps_in = homepath + "surface_Energy\\fe_bcc100_reaxff_minimize.in",
                               lammps_trj = homepath + "surface_Energy\\fe_bcc100_reaxff_minimize.lammpstrj")

In [ ]:
# Running calculations
!lmp_serial -in surface_Energy\fe_bcc100_reaxff_minimize.in -log surface_Energy\fe_bcc100_reaxff_minimize.log

In [8]:
# Reading output
Fe_bcc100_calc.read_lammps_log(lammps_log = homepath + "surface_Energy\\fe_bcc100_reaxff_minimize.log")
Fe_bcc100_data = Fe_bcc100_calc.thermo_content[-1]

In [13]:
Fe_bcc100_data = Fe_bcc100_calc.thermo_content[-1]

# Surface energy in Kcal/(mol A^2)
Fe_bcc100_SE_real = (Fe_bcc100_data['pe']/Fe_bcc100_data['atoms'] - Fe_bulk_data['pe']/Fe_bulk_data['atoms'])/(2*(Fe_bcc100_data['vol']/Fe_bcc100_data['lz']/Fe_bcc100_data['atoms']))

# Converting to J/m^2
Fe_bcc100_SE_SI = Fe_bcc100_SE_real * 4184 / cst.Avogadro * 10**20
print(Fe_bcc100_SE_SI)

2.0692676658333244


# bcc(111) Fe

In [9]:
# Creating simulation box
Fe_bcc111 = bcc111('Fe', size=(1,1,128), vacuum=10.0)

# Setting up simulation input
Fe_bcc111_calc = LAMMPS(tmp_dir = homepath + "temp", 
                      parameters = Fe_slab_reaxff_params,
                      keep_tmp_files=True, no_data_file=True)
Fe_bcc111.set_calculator(Fe_bcc111_calc)
Fe_bcc111_calc.atoms = Fe_bcc111.copy()
Fe_bcc111_calc.prism = Prism(Fe_bcc111_calc.atoms.get_cell())

# Writing input file
Fe_bcc111_calc.write_lammps_in(lammps_in = homepath + "surface_Energy\\fe_bcc111_reaxff_minimize.in",
                               lammps_trj = homepath + "surface_Energy\\fe_bcc111_reaxff_minimize.lammpstrj")

In [ ]:
# Running calculations
!lmp_serial -in surface_Energy\fe_bcc111_reaxff_minimize.in -log surface_Energy\fe_bcc111_reaxff_minimize.log

In [10]:
# Reading output
Fe_bcc111_calc.read_lammps_log(lammps_log = homepath + "surface_Energy\\fe_bcc111_reaxff_minimize.log")

In [ ]:
Fe_bcc111_calc.thermo_content[-1]

In [12]:
Fe_bcc111_data = Fe_bcc111_calc.thermo_content[-1]

# Surface energy in Kcal/(mol A^2)
Fe_bcc111_SE_real = (Fe_bcc111_data['pe']/Fe_bcc111_data['atoms'] - Fe_bulk_data['pe']/Fe_bulk_data['atoms'])/(2*(Fe_bcc111_data['vol']/Fe_bcc111_data['lz']/Fe_bcc111_data['atoms']))

# Converting to J/m^2
Fe_bcc111_SE_SI = Fe_bcc111_SE_real * 4184 / cst.Avogadro * 10**20
print(Fe_bcc111_SE_SI)

2.6762248435373914


# etc

In [ ]:

Fe_var_reaxff_thermo_contents = []
Fe_var_reaxff_thermo_contents_label = []

for thickness in range(2, 60, 4):
    Fe_var_reaxff_thermo_contents_label.append(thickness)
    Fe_var_reaxff_slab = bcc110('Fe', size=(1,1,thickness), vacuum=10.0)
    
    Fe_var_reaxff_slab_calc = LAMMPS(tmp_dir = homepath + "temp", 
                              parameters = fe_slab_lammps_params,
                              keep_tmp_files=True, no_data_file=True)
    
    Fe_slab.set_calculator(Fe_var_reaxff_slab_calc)
    Fe_var_reaxff_slab_calc.atoms = Fe_var_reaxff_slab.copy()
    Fe_var_reaxff_slab_calc.prism = Prism(Fe_var_reaxff_slab_calc.atoms.get_cell())
    
    Fe_var_reaxff_slab_calc.write_lammps_in(lammps_in = homepath + "input\\fe_var_slab_reaxff_minimize.in",
                                     lammps_trj = homepath + "output\\fe_var_slab_reaxff_minimize.lammpstrj")
    
    !lmp_serial -in input\fe_var_slab_reaxff_minimize.in -log output\fe_var_slab_reaxff_minimize.log
    
    Fe_var_reaxff_slab_calc.read_lammps_log(lammps_log = homepath + "output\\fe_var_slab_reaxff_minimize.log")
    Fe_var_reaxff_thermo_contents.append(Fe_var_reaxff_slab_calc.thermo_content)